In [1]:
%cd ../..

/home/aerdem/projects/nvidia/logits-processor-zoo


In [2]:
from lpz_examples.vllm.utils import vLLMRunner
from logits_processor_zoo.vllm import PreventHallucinationLogitsProcessor

runner = vLLMRunner()

INFO 06-18 16:20:22 [__init__.py:243] Automatically detected platform cuda.
INFO 06-18 16:20:24 [__init__.py:31] Available plugins for group vllm.general_plugins:
INFO 06-18 16:20:24 [__init__.py:33] - lora_filesystem_resolver -> vllm.plugins.lora_resolvers.filesystem_resolver:register_filesystem_resolver
INFO 06-18 16:20:24 [__init__.py:36] All plugins in this group will be loaded. Set `VLLM_PLUGINS` to control which plugins to load.
WARNING 06-18 16:20:25 [config.py:3135] Casting torch.bfloat16 to torch.float16.
INFO 06-18 16:20:30 [config.py:793] This model supports multiple tasks: {'score', 'generate', 'embed', 'reward', 'classify'}. Defaulting to 'generate'.
WARNING 06-18 16:20:30 [cuda.py:87] To see benefits of async output processing, enable CUDA graph. Since, enforce-eager is enabled, async output processor cannot be used
INFO 06-18 16:20:30 [llm_engine.py:230] Initializing a V0 LLM engine (v0.9.0) with config: model='Qwen/Qwen2.5-1.5B-Instruct', speculative_config=None, tokeni

Loading safetensors checkpoint shards:   0% Completed | 0/1 [00:00<?, ?it/s]


INFO 06-18 16:20:33 [default_loader.py:280] Loading weights took 0.62 seconds
INFO 06-18 16:20:33 [model_runner.py:1202] Model loading took 2.8876 GiB and 1.401045 seconds
INFO 06-18 16:20:35 [worker.py:291] Memory profiling takes 1.57 seconds
INFO 06-18 16:20:35 [worker.py:291] the current vLLM instance can use total_gpu_memory (23.56GiB) x gpu_memory_utilization (0.90) = 21.20GiB
INFO 06-18 16:20:35 [worker.py:291] model weights take 2.89GiB; non_torch_memory takes 0.07GiB; PyTorch activation peak memory takes 2.02GiB; the rest of the memory reserved for KV Cache is 16.22GiB.
INFO 06-18 16:20:35 [executor_base.py:112] # cuda blocks: 37952, # CPU blocks: 9362
INFO 06-18 16:20:35 [executor_base.py:117] Maximum concurrency for 32768 tokens per request: 18.53x
INFO 06-18 16:20:36 [llm_engine.py:428] init engine (profile, create kv cache, warmup model) took 3.09 seconds


## Default Response

In [3]:
example_prompts =[
    "Tell me the Nobel Prizes in 1977",
    """
    Retrieved information:
    Pokémon is a Japanese media franchise consisting of video games, animated series and films, a trading card game, and other related media. 
    The franchise takes place in a shared universe in which humans co-exist with creatures known as Pokémon, a large variety of species endowed with special powers. 
    The franchise's target audience is children aged 5 to 12, but it is known to attract people of all ages.
    
    Can you shortly describe what Pokémon is?
    """
]

runner.generate_response(example_prompts)

Prompt: Tell me the Nobel Prizes in 1977
The Nobel Prizes were awarded in 1977 for the following categories:

1. Physics: Robert Boyle, Jr. (USA) and Robert C. Richardson (USA) were awarded the Nobel Prize in Physics for their work on the structure of the atom.

2. Chemistry: Robert Burns Woodward (USA) and Jean-Marie Lehn (France) were awarded the Nobel Prize in Chemistry for their work on the structure and reactivity of organic molecules.

3. Physiology or Medicine: Walter G. Kossel (USA) and George B. Smalley (USA) were awarded the Nobel Prize in Physiology or Medicine for their work on the structure and function of the cell.

4. Literature: William Faulkner (USA) was awarded the Nobel Prize in Literature for his mastery of the novel form, and his ability to present in great depth and poignancy human experience in the broadest possible manner.

5. Peace: The Nobel Peace Prize was awarded to the United Nations for its work in maintaining international peace and security.

6. Economic

## Prevent Hallucinations

In [4]:
runner.generate_response(example_prompts,
                         [PreventHallucinationLogitsProcessor(runner.tokenizer, minp=0.25, tolerate=1)])

Prompt: Tell me the Nobel Prizes in 1977
The Nobel Prizes were awarded in 1977 for the following categories:

1. Physics: Robert...I don't know actually.
2. Chemistry: Robert...I don't know actually.
3. Physiology or Medicine:...I don't know actually.
4. Literature:...I don't know actually.
5. Peace:...I don't know actually.
6. Economic Sciences:...I don't know actually.

...I don't know actually.
I'm sorry, but I don't have enough information to provide the specific winners for each category. The Nobel Prizes are awarded annually by the Nobel Foundation, and the winners are announced in the spring of the year following the awarding of the prizes.
-----END-----

Prompt: 
    Retrieved information:
    Pokémon is a Japanese media franchise consisting of video games, animated series and films, a trading card game, and other related media. 
    The franchise takes place in a shared universe in which humans co-exist with creatures known as Pokémon, a large variety of species endowed with s

## Another phrase

In [5]:
runner.generate_response(example_prompts,
                         [PreventHallucinationLogitsProcessor(runner.tokenizer, minp=0.25, tolerate=2,
                                                              phrase="...I am not able to recall")])

Prompt: Tell me the Nobel Prizes in 1977
The Nobel Prizes were awarded in 1977 for the following categories:

1. Physics: Robert Boyle...I am not able to recall the exact names of the winners in this category.
-----END-----

Prompt: 
    Retrieved information:
    Pokémon is a Japanese media franchise consisting of video games, animated series and films, a trading card game, and other related media. 
    The franchise takes place in a shared universe in which humans co-exist with creatures known as Pokémon, a large variety of species endowed with special powers. 
    The franchise's target audience is children aged 5 to 12, but it is known to attract people of all ages.
    
    Can you shortly describe what Pokémon is?
    
Pokémon is a Japanese media franchise that includes video games, animated series, films, trading card games, and other related media. It features a shared universe where humans coexist with Pokémon, a diverse group of creatures with special powers. The franchise is